In [63]:
import json
import pickle

In [65]:
questions = []
contexts = []
examples = []
labels = []
id2idx = {}
files = ["dataset/train-v2.0.json",  "dataset/dev-v2.0.json"]
for file in files:
    with open(file, 'r') as handle:
        jdata = json.load(handle)
        data = jdata['data']
    for i in range(len(data)):
        section = data[i]['paragraphs']
        for sec in section:
            context = sec['context']
            contexts.append(context)
            qas = sec['qas']
            for j in range(len(qas)):
                question = qas[j]['question']
                is_imp = qas[j]['is_impossible']
                qid = qas[j]['id']
                questions.append(question)
                labels.append(is_imp)
                examples.append((len(contexts)-1, len(questions)-1))
                id2idx[qid] = len(questions)-1

In [66]:
ftmp = "qparts/copy_parts2/parsed_qs_labels%s.pkl"
parses = {}
for i in range(24):
    fname = ftmp % str(i)
    with open(fname, "rb") as f:
        data = pickle.load(f)
        parses.update(data)

In [67]:
list(parses.keys())[1]

'56ddde6b9a695914005b9629'

In [68]:
qid = '56ddde6b9a695914005b9629'
i = id2idx[qid]
print(questions[i])
print(parses[qid])

When were the Normans in Normandy?
[(['norman', '##s'], ('Normans', 'NNPS')), (['normandy'], ('Normandy', 'NNP')), (['when'], ('When', 'WHADVP'))]


# TODO
- Words not included in parse
- % of all words caught
- % of questions 100% covered
- split by answerable and unanswerable

In [69]:
from pytorch_pretrained_bert.tokenization import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

In [70]:
qtokens = []
for q in questions:
    qtokens.append(tokenizer.tokenize(q))

In [71]:
from collections import Counter

In [77]:
# stats
missed_words = Counter()
missed_words_ans = Counter()
missed_words_unans = Counter()

coverage = [0]*21
coverage_ans = [0]*21
coverage_unans = [0]*21

for qid, qparse in parses.items():
    idx = id2idx[qid]
    q = questions[idx]
    label = labels[idx]
    toks = qtokens[idx]
    all_toks = set(toks)
    parsed_toks = set()
    for chunk in qparse:
        (pt, (_,_)) = chunk
        parsed_toks.update(pt)
    cov_pct = float(len(parsed_toks)) / len(all_toks)
    bucket = int(cov_pct * 100 // 5)
    if bucket >= len(coverage):
        bucket = 20
        print("overflow")
    coverage[bucket] += 1
    if label == True:
        coverage_unans[bucket] += 1
    else:
        coverage_ans[bucket] += 1
    for tok in all_toks:
        if tok not in parsed_toks:
            missed_words[tok] += 1
            if label == True:
                missed_words_unans[tok] += 1
            else:
                missed_words_ans[tok] += 1

overflow
overflow
overflow
overflow


In [78]:
import numpy as np

In [79]:
for i, num in enumerate(coverage / np.sum(coverage)):
    print(i*5,"%=>","%.2f"%(num*100))

0 %=> 0.00
5 %=> 0.00
10 %=> 0.00
15 %=> 0.00
20 %=> 0.02
25 %=> 0.08
30 %=> 0.22
35 %=> 0.44
40 %=> 2.03
45 %=> 1.10
50 %=> 9.01
55 %=> 9.26
60 %=> 17.57
65 %=> 13.98
70 %=> 14.29
75 %=> 13.83
80 %=> 10.49
85 %=> 4.30
90 %=> 3.02
95 %=> 0.07
100 %=> 0.31


In [80]:
for i, num in enumerate(coverage_ans / np.sum(coverage_ans)):
    print(i*5,"%=>","%.2f"%(num*100))

0 %=> 0.00
5 %=> 0.00
10 %=> 0.00
15 %=> 0.00
20 %=> 0.01
25 %=> 0.07
30 %=> 0.21
35 %=> 0.41
40 %=> 2.01
45 %=> 1.12
50 %=> 9.22
55 %=> 9.45
60 %=> 17.81
65 %=> 14.30
70 %=> 14.23
75 %=> 13.70
80 %=> 10.31
85 %=> 4.14
90 %=> 2.81
95 %=> 0.08
100 %=> 0.09


In [81]:
for i, num in enumerate(coverage_unans / np.sum(coverage_unans)):
    print(i*5,"%=>","%.2f"%(num*100))

0 %=> 0.00
5 %=> 0.00
10 %=> 0.00
15 %=> 0.00
20 %=> 0.03
25 %=> 0.09
30 %=> 0.22
35 %=> 0.50
40 %=> 2.06
45 %=> 1.05
50 %=> 8.61
55 %=> 8.92
60 %=> 17.11
65 %=> 13.38
70 %=> 14.39
75 %=> 14.07
80 %=> 10.80
85 %=> 4.60
90 %=> 3.40
95 %=> 0.04
100 %=> 0.72


In [82]:
missed_words.most_common(25)

[('?', 140724),
 ('the', 63964),
 ('of', 27005),
 ('was', 25118),
 ('is', 25036),
 ('did', 24130),
 ('to', 23356),
 ('in', 20449),
 ('a', 14102),
 ('for', 8071),
 ('are', 7865),
 ('does', 7255),
 (',', 6663),
 ('were', 5658),
 ('do', 5096),
 ('as', 4900),
 ('with', 4471),
 ('have', 4367),
 ('on', 4175),
 ('from', 4147),
 ('by', 3943),
 ('and', 3726),
 ('be', 3624),
 ('an', 3326),
 ('used', 2928)]

In [83]:
missed_words_ans.most_common(25)

[('?', 91666),
 ('the', 42845),
 ('of', 18141),
 ('was', 17199),
 ('is', 16726),
 ('did', 16380),
 ('to', 15400),
 ('in', 13091),
 ('a', 9261),
 ('for', 5512),
 (',', 5157),
 ('are', 5080),
 ('does', 4732),
 ('were', 3967),
 ('as', 3308),
 ('do', 3137),
 ('with', 3064),
 ('on', 2857),
 ('have', 2765),
 ('from', 2686),
 ('by', 2600),
 ('and', 2497),
 ('be', 2437),
 ('an', 2163),
 ('used', 1896)]

In [84]:
missed_words_unans.most_common(25)

[('?', 49058),
 ('the', 21119),
 ('of', 8864),
 ('is', 8310),
 ('to', 7956),
 ('was', 7919),
 ('did', 7750),
 ('in', 7358),
 ('a', 4841),
 ('are', 2785),
 ('for', 2559),
 ('does', 2523),
 ('do', 1959),
 ('were', 1691),
 ('have', 1602),
 ('as', 1592),
 (',', 1506),
 ('from', 1461),
 ('with', 1407),
 ('by', 1343),
 ('on', 1318),
 ('and', 1229),
 ('be', 1187),
 ('has', 1164),
 ('an', 1163)]

In [85]:
import spacy
nlp = spacy.load("en_core_web_sm")


In [86]:
for tok, count in missed_words.most_common(200):
    if tok not in nlp.vocab or (not nlp.vocab[tok].is_stop and not nlp.vocab[tok].is_punct):
        print(tok,count)

didn 394
wasn 332
responsible 293
able 267
doesn 265
isn 262
use 252
like 227
order 218
end 203
t 184
##s 165
don 162
present 138
aren 130
longer 129
work 126
non 122
common 119
start 115
available 108
open 107
weren 105
similar 103
outside 99
rank 98
popular 94
different 91
important 90
near 87
un 84
cause 82
help 78
claim 75
form 72
state 69
likely 69
increase 68
rule 63
kind 62
necessary 59
couldn 52
unable 50
mean 48
lead 48
possible 47
support 47
change 46
title 46
despite 45
inside 45
##e 45
hold 43
better 43
main 42
date 42
famous 41
produce 41
easy 40
lack 40


In [87]:
for tok, count in missed_words_ans.most_common(200):
    if tok not in nlp.vocab or (not nlp.vocab[tok].is_stop and not nlp.vocab[tok].is_punct):
        print(tok,count)

responsible 209
use 174
able 172
like 161
order 144
end 119
present 112
##s 110
work 86
open 85
available 75
rank 73
common 72
similar 70
different 66
start 65
important 63
popular 59
near 56
claim 55
cause 53
likely 53
form 51
didn 50
outside 49
increase 47
kind 46
state 44
rule 43
doesn 42
t 42
don 39
help 39
title 39
necessary 37
hold 36
date 35
despite 34
better 32
change 32
main 31
un 31
famous 31
mean 31
nickname 31
##e 30
lead 30
possible 30
job 30
air 30
offer 29
produce 29
couldn 29
travel 28
profession 28
share 28
partner 28
best 27
experience 27
support 26


In [88]:
for tok, count in missed_words_unans.most_common(200):
    if tok not in nlp.vocab or (not nlp.vocab[tok].is_stop and not nlp.vocab[tok].is_punct):
        print(tok,count)

didn 344
wasn 309
isn 237
doesn 223
t 142
don 123
aren 110
non 104
longer 103
weren 97
able 95
responsible 84
end 84
use 78
order 74
like 66
##s 55
un 53
outside 50
start 50
common 47
work 40
help 39
popular 35
available 33
similar 33
near 31
unable 31
cause 29
easy 27
important 27
present 26
state 25
different 25
rank 25
smallest 24
inside 24
couldn 23
di 23
necessary 22
open 22
unknown 22
increase 21
form 21
support 21
worth 21
dependent 20
rule 20
wouldn 20
claim 20
##vent 18
impossible 18
lead 18
hasn 18
separate 18
mean 17
possible 17
kind 16
unpopular 16
lack 16
likely 16
s 15
difficult 15
uncommon 15
##e 15


# TODO: Fix up all the "isn't doesn't wasn't couldn't don't etc. above"

# What do sentences look like in different coverage zones

In [90]:
# stats
missed_words = Counter()
missed_words_ans = Counter()
missed_words_unans = Counter()

coverage = [0]*21
coverage_ans = [0]*21
coverage_unans = [0]*21

coverage_examples = {}

for qid, qparse in parses.items():
    idx = id2idx[qid]
    q = questions[idx]
    label = labels[idx]
    toks = qtokens[idx]
    all_toks = set(toks)
    parsed_toks = set()
    for chunk in qparse:
        (pt, (_,_)) = chunk
        parsed_toks.update(pt)
    cov_pct = float(len(parsed_toks)) / len(all_toks)
    bucket = int(cov_pct * 100 // 5)
    if bucket > 20:
        bucket = 20
    if bucket not in coverage_examples:
        coverage_examples[bucket] = []
    coverage_examples[bucket].append(qid)
    coverage[bucket] += 1
    if label == True:
        coverage_unans[bucket] += 1
    else:
        coverage_ans[bucket] += 1
    for tok in all_toks:
        if tok not in parsed_toks:
            missed_words[tok] += 1
            if label == True:
                missed_words_unans[tok] += 1
            else:
                missed_words_ans[tok] += 1

In [91]:
len(coverage_examples)

21

In [92]:
import random

In [94]:
for key in range(21):
    val = coverage_examples[key]
    print(key*5, "%")
    if len(val) < 5:
        print("Too few samples... skipping.")
        continue
    for qid in random.sample(val, 5):
        idx = id2idx[qid]
        print(labels[idx], "=>",questions[idx])
        print(parses[qid])
        print("~"*20)
    print("#"*20)

0 %
Too few samples... skipping.
5 %
Too few samples... skipping.
10 %
Too few samples... skipping.
15 %
False => The goal is to get how  many residents to sign up?
[(['goal'], ('goal', 'NN')), (['sign'], ('sign', 'VB'))]
~~~~~~~~~~~~~~~~~~~~
False => What did the quake measure?
[(['what'], ('What', 'WHNP'))]
~~~~~~~~~~~~~~~~~~~~
False => How idealistic were samurai?
[(['samurai'], ('samurai', 'NN'))]
~~~~~~~~~~~~~~~~~~~~
True => What is either allophonic or a cluster?
[(['cluster'], ('cluster', 'NN')), (['what'], ('What', 'WHNP'))]
~~~~~~~~~~~~~~~~~~~~
True => The flow within what is multidirectional?
[(['flow'], ('flow', 'NN')), (['what'], ('what', 'WHNP'))]
~~~~~~~~~~~~~~~~~~~~
####################
20 %
False => Bermuda is closest (in nautical miles) to which US state?
[(['nautical', 'miles'], ('nautical miles', 'NP')), (['bermuda'], ('Bermuda', 'NN'))]
~~~~~~~~~~~~~~~~~~~~
False => Did that used to have a different name?
[(['different', 'name'], ('different name', 'NP')), ([], ('',

In [60]:
for q in questions:
    if q == 'd':
        print(q)

d
